# Using TensorBoard in Kaggle Kernels

**Note:** TensorBoard requires a running kernel, so its output will only be available in an editor session.

In [ ]:
# Load the extension and start TensorBoard

%load_ext tensorboard.notebook
%tensorboard --logdir logs

In [ ]:
import pickle
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time
import numpy as np


In [ ]:
pickle_in = open("/kaggle/input/dataset-1/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("/kaggle/input/dataset-1/y.pickle","rb")
y = pickle.load(pickle_in)

In [ ]:
X = X/255.0
y=np.array(y)

print(X.shape, X.dtype)
print(y.shape, y.dtype)

In [ ]:
dense_layers=[0]
layer_sizes=[64]
conv_layers=[3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            cnn_model = "{}-conv-{}-nodes-{}-dense-{}-dropout1".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(cnn_model)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(cnn_model))
            #opt = tf.keras.optimizers.Adam(lr=0.0001)
            

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=128,
                      epochs=20,
                      validation_split=0.3,
                      callbacks=[tensorboard])
            
            
    

In [ ]:
tf.keras.utils.plot_model(
    model
)

In [ ]:
model.save('64x3-CNN.model')

In [ ]:
%load_ext tensorboard.notebook
%tensorboard --logdir logs